In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

1. **Recency**: Refers to how recently the customer made a purchase. This metric helps determine whether a customer is likely to need re-engagement. A high recency value indicates the customer may require re-engagement soon.
1. **Total_baskets**: Refers to the total number of transactions (baskets) made by a customer. A high value indicates that the customer makes purchases frequently, whereas a low value suggests that the customer tends to make one-time or infrequent purchases.
1. **Total_spend_money**: The total amount of (positive net_value_dkk) money spent by a customer.
1. **Total_refund_money**: The total amount of (negative quantity) money refunded to the customer.
1. **Total_net_revenue**: The total net amount of money spent by the customer, accounting for any refunds or adjustments [sum of net_value_dkk].
1. **Average_basket_spend_value**: The average basket value of a customer [sum of positive net_value_dkk/distinct count of order_id].
1. **Total_items_purchased**: Represents the total number of items a customer has purchased [sum of positive quantity]. A high value indicates that the customer has made a significant volume of purchases.
1. **Total_distinct_items_purchased**: Indicates the total number of unique items a customer has purchased. This metric reflects the diversity in the customer's purchasing preferences.
1. **Total_returned_items**: Represents the total number of items a customer has returned [sum of negative quantity]. A high value indicates that the customer has returned a significant volume of items.
1. **Favourite_day_of_week**: Refers to the day of the week when the customer most often makes their purchases.
1. **Favourite_metal**: Refers to the type of metal most frequently purchased by the customer [one hot encoded].
1. **prefer_theme**: Indicator whether the customer prefers theme or not [0-1 encoded].
1. **Favourite_store_type**: Refers to the type of store the customer prefers for their purchases. [one hot encoded]

In [2]:
orders = pd.read_csv("./Data/order.csv", header=0, index_col=0)
products = pd.read_csv("./Data/product.csv", header=0, index_col=0, 
    dtype={
        'category': 'str',
        'collection': 'str',
        'color': 'str',
        'metal': 'str',  # Specify types explicitly if needed
    }
)

In [3]:
final_df = orders.merge(products, on="product_id_with_size", how="left")
final_df['day_of_week'] = pd.to_datetime(final_df['sellout_date']).dt.dayofweek
final_df['day_of_year'] = pd.to_datetime(final_df['sellout_date']).dt.dayofyear

In [4]:
df = final_df
df["quantity"] = df["quantity"].abs()
df['day_of_week'] = pd.to_datetime(df['sellout_date']).dt.dayofweek
df = df.groupby(['consumer_id', 'day_of_week']).agg(
    quantity_volume=pd.NamedAgg(column="quantity", aggfunc="sum"),
)
df = df.reset_index()
df = df.loc[df.groupby('consumer_id')['quantity_volume'].idxmax()]
df

,consumer_id,day_of_week,quantity_volume
0,00002048671f53146b94bdb6878816f7,5,2
1,00002c39b6e71b2d67dfa58dd240a4e8,4,2
2,0000866bfcf45b80db3df4181010fad0,3,2
3,00009e5c1eb35cca8166462dcce55312,0,1
4,0000e3a35d2889933909d0d1a858bb16,2,1
...,...,...,...
333105,ffff2cd3c3163b7ce8072d96da01beb2,2,1
333106,ffff31eca55965dfb78ad60b2c6992d8,5,1
333107,ffff65a8aac93617933aedc11b922a1b,5,1
333108,ffff70d2c24296656d8c50f16ff79783,5,2


In [5]:
import feature_egnineering as F
# from datetime import datetime

In [6]:
functions = [
    # F.compute_recency,
    F.compute_total_baskets,
    F.compute_total_spend_money,
    F.compute_total_refund_money,
    F.compute_total_net_revenue,
    F.compute_average_basket_spend,
    F.compute_total_items_purchased,
    F.compute_total_returned_items,
    F.compute_favourite_day_of_week,
    F.compute_favourite_metal,
    F.compute_favourite_store_type,
    # save_features_to_parquet
]

In [7]:
F.compute_recency(final_df, "2024-01-01")

,consumer_id,recency
0,00002048671f53146b94bdb6878816f7,30
1,00002c39b6e71b2d67dfa58dd240a4e8,10
2,0000866bfcf45b80db3df4181010fad0,25
3,00009e5c1eb35cca8166462dcce55312,266
4,0000e3a35d2889933909d0d1a858bb16,327
...,...,...
319069,ffff2cd3c3163b7ce8072d96da01beb2,159
319070,ffff31eca55965dfb78ad60b2c6992d8,86
319071,ffff65a8aac93617933aedc11b922a1b,289
319072,ffff70d2c24296656d8c50f16ff79783,233


In [11]:
len([
    "compute_recency",
    "compute_total_baskets",
    "compute_total_spend_money",
    "compute_total_refund_money",
    "compute_total_net_revenue",
    "compute_average_basket_spend",
    "compute_total_items_purchased",
    "compute_total_distinct_items_purchased",
    "compute_total_returned_items",
    "compute_favourite_day_of_week",
    "compute_favourite_metal",
    "compute_favourite_store_type",
    "compute_prefer_theme",
])

13

In [8]:
for function in functions:
    display(function(final_df))


,consumer_id,total_baskets
0,00002048671f53146b94bdb6878816f7,1
1,00002c39b6e71b2d67dfa58dd240a4e8,1
2,0000866bfcf45b80db3df4181010fad0,1
3,00009e5c1eb35cca8166462dcce55312,1
4,0000e3a35d2889933909d0d1a858bb16,1
...,...,...
319069,ffff2cd3c3163b7ce8072d96da01beb2,1
319070,ffff31eca55965dfb78ad60b2c6992d8,1
319071,ffff65a8aac93617933aedc11b922a1b,1
319072,ffff70d2c24296656d8c50f16ff79783,1


,consumer_id,total_spend_money
0,00002048671f53146b94bdb6878816f7,973.91
1,00002c39b6e71b2d67dfa58dd240a4e8,973.12
2,0000866bfcf45b80db3df4181010fad0,399.45
3,00009e5c1eb35cca8166462dcce55312,546.25
4,0000e3a35d2889933909d0d1a858bb16,242.06
...,...,...
319069,ffff2cd3c3163b7ce8072d96da01beb2,302.44
319070,ffff31eca55965dfb78ad60b2c6992d8,350.48
319071,ffff65a8aac93617933aedc11b922a1b,545.85
319072,ffff70d2c24296656d8c50f16ff79783,645.61


,consumer_id,total_refund_money


,consumer_id,total_refund_money
0,00002048671f53146b94bdb6878816f7,973.91
1,00002c39b6e71b2d67dfa58dd240a4e8,973.12
2,0000866bfcf45b80db3df4181010fad0,399.45
3,00009e5c1eb35cca8166462dcce55312,546.25
4,0000e3a35d2889933909d0d1a858bb16,242.06
...,...,...
319069,ffff2cd3c3163b7ce8072d96da01beb2,302.44
319070,ffff31eca55965dfb78ad60b2c6992d8,350.48
319071,ffff65a8aac93617933aedc11b922a1b,545.85
319072,ffff70d2c24296656d8c50f16ff79783,645.61


,consumer_id,spend,baskets,average_basket_spend
0,00002048671f53146b94bdb6878816f7,973.91,1,973.91
1,00002c39b6e71b2d67dfa58dd240a4e8,973.12,1,973.12
2,0000866bfcf45b80db3df4181010fad0,399.45,1,399.45
3,00009e5c1eb35cca8166462dcce55312,546.25,1,546.25
4,0000e3a35d2889933909d0d1a858bb16,242.06,1,242.06
...,...,...,...,...
319069,ffff2cd3c3163b7ce8072d96da01beb2,302.44,1,302.44
319070,ffff31eca55965dfb78ad60b2c6992d8,350.48,1,350.48
319071,ffff65a8aac93617933aedc11b922a1b,545.85,1,545.85
319072,ffff70d2c24296656d8c50f16ff79783,645.61,1,645.61


,consumer_id,total_items
0,00002048671f53146b94bdb6878816f7,2
1,00002c39b6e71b2d67dfa58dd240a4e8,2
2,0000866bfcf45b80db3df4181010fad0,2
3,00009e5c1eb35cca8166462dcce55312,1
4,0000e3a35d2889933909d0d1a858bb16,1
...,...,...
319069,ffff2cd3c3163b7ce8072d96da01beb2,1
319070,ffff31eca55965dfb78ad60b2c6992d8,1
319071,ffff65a8aac93617933aedc11b922a1b,1
319072,ffff70d2c24296656d8c50f16ff79783,2


,consumer_id,total_items


,consumer_id,day_of_week,quantity_volume
0,00002048671f53146b94bdb6878816f7,5,2
1,00002c39b6e71b2d67dfa58dd240a4e8,4,2
2,0000866bfcf45b80db3df4181010fad0,3,2
3,00009e5c1eb35cca8166462dcce55312,0,1
4,0000e3a35d2889933909d0d1a858bb16,2,1
...,...,...,...
333105,ffff2cd3c3163b7ce8072d96da01beb2,2,1
333106,ffff31eca55965dfb78ad60b2c6992d8,5,1
333107,ffff65a8aac93617933aedc11b922a1b,5,1
333108,ffff70d2c24296656d8c50f16ff79783,5,2


,consumer_id,metal,quantity_volume
0,00002048671f53146b94bdb6878816f7,4,2
1,00002c39b6e71b2d67dfa58dd240a4e8,4,2
2,0000866bfcf45b80db3df4181010fad0,4,2
3,00009e5c1eb35cca8166462dcce55312,4,1
4,0000e3a35d2889933909d0d1a858bb16,4,1
...,...,...,...
342013,ffff2cd3c3163b7ce8072d96da01beb2,4,1
342014,ffff31eca55965dfb78ad60b2c6992d8,4,1
342015,ffff65a8aac93617933aedc11b922a1b,4,1
342016,ffff70d2c24296656d8c50f16ff79783,4,1


,consumer_id,store_type,quantity_volume
0,00002048671f53146b94bdb6878816f7,Concept Store,2
1,00002c39b6e71b2d67dfa58dd240a4e8,Concept Store,2
2,0000866bfcf45b80db3df4181010fad0,Online,2
3,00009e5c1eb35cca8166462dcce55312,Concept Store,1
4,0000e3a35d2889933909d0d1a858bb16,Online,1
...,...,...,...
323231,ffff2cd3c3163b7ce8072d96da01beb2,Concept Store,1
323232,ffff31eca55965dfb78ad60b2c6992d8,Concept Store,1
323233,ffff65a8aac93617933aedc11b922a1b,Concept Store,1
323234,ffff70d2c24296656d8c50f16ff79783,Concept Store,2
